<a href="https://colab.research.google.com/github/Samadkhani/ImageRecommendation/blob/main/CompleteCode_RLRS_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

# Importing necessary libraries
import datetime
import numpy as np
from scipy import io
import scipy.io as spio
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from collections import deque
import random
from sklearn.linear_model import Lasso

import pandas as pd
from matplotlib import pyplot as plt

from sklearn.metrics import mean_squared_error
from tensorflow.python.keras.layers import Concatenate , Dense, Input, Dropout, Add, Activation
#from tensorflow.keras.layers import BatchNormalization
from tensorflow.python.keras import Model

import matplotlib.pyplot as plt
from datetime import datetime
#from buffer import BasicBuffer_a,BasicBuffer_b
from sys import exit
import random
from collections import deque

# Changing directory to the location of the data
%cd /content/gdrive/MyDrive/TestTrain_3Features

# Loading the data
matTrain = io.loadmat('Train_user__1.mat')
matTest = io.loadmat('Test_user__1.mat')
mTrainData = matTrain['Train_user__1']
mTestData = matTest['Test_user__1']

mTrain = np.zeros((1100, 108))
mTest = np.zeros((1100, 108))
print(mTrain.shape)
print(mTest.shape)

mTrain[:,0:107] = mTrainData[:,0:107]
mTest[:,0:107] = mTestData[:,0:107]

mTrain[:,-1] = mTrainData[:,-1]
mTest[:,-1] = mTestData[:,-1]

print(mTrain.shape)
print(mTest.shape)


UserID = 1
num_Features = 107



bz = 10 # batch_size

start_step = 1000 # start_steps

n_t_e = 400 # num_train_episodes

mel = 100

replay_sizeee = n_t_e * mel

lr = 1e-3
gam=0.5
s_r = 0.5

K_items =1

hidden_sizes_1=(256,128)
hidden_sizes_2=(100,64, 32)

def CalculateReward(ImgData, UserID):
  Reward = 0
  #numUsers = 20    ########################################### numUsers #######################################
  allReward = []
  numData , m1 = ImgData.shape
  for i in range(numData):
    classData = ImgData[i , -1]
    if classData == UserID:
      Reward = 1
    else:
      Reward = -1
    allReward.append(Reward)
  #newImgData = np.hstack((ImgData,allReward))
  return allReward

RewardDataaaaTrain = CalculateReward(mTrain , UserID)
RewardDataaaaTest = CalculateReward(mTest , UserID)
RewardDataTrain = np.array(RewardDataaaaTrain)
RewardDataTest = np.array(RewardDataaaaTest)


# Concat Reward and Data

def concatReward_Data(Data, RewardData):
  numData , m1 = Data.shape
  DatawithReward = np.zeros((numData,m1+1))

  for i in range(numData):
    t = Data[i,:]  #     ERORR      t = mTrain[i,:]
    Rt = RewardData[i]  #    ERROR      Rt = RewardDataTrain[i]
    DatawithReward[i,:] = np.hstack((t,Rt))

  return DatawithReward

#--------------------------------
Data_Reward_Train = concatReward_Data(mTrain, RewardDataTrain)
Data_Reward_Test = concatReward_Data(mTest, RewardDataTest)

# reset initial state
def resetFun(Data , UserID, state_dim):
  numData , m1 = Data.shape
  init_state = []
  indx = 0
  for i in range(numData):
    if (Data[i , -1] == UserID):
      init_state = Data[i,:state_dim]
      indx = i
      break

  return init_state, indx


def representation_state(pos_state):
  #print("pos_state")
  #print(pos_state)
  size_pos = pos_state.size
  if size_pos>num_Features :
    mean_pos = np.mean(pos_state, axis = 0)
  elif size_pos== num_Features:
    mean_pos = pos_state

  return mean_pos


from numpy import dot
from scipy.special import softmax

def selfAttention_representation(aa, pos_state):
  size_pos = pos_state.size
  pos_state_array = np.array(pos_state)
  n , m = pos_state_array.shape
  #print("pos_state_array", pos_state_array.shape)
  #print("n=", n)
  #print("m=", m)

  scoresj = np.zeros((n,))
  attention = np.zeros((m,))

  for j in range (n):
    scoresj[j] = np.dot(pos_state_array[j,:] , aa)

  #print("scoresj", scoresj.shape)
  #print("aa", aa.shape[0])
  # computing the weights by a softmax operation

  weights = softmax(scoresj)

  # computing the attention by a weighted sum of the value vectors
  for j in range (n):
    pos_state_array[j,:] = (np.dot(weights[j] , pos_state_array[j,:]))

  attention = np.sum(pos_state_array, axis=0)
  return attention


def stepFun( a , Inx_I_2 , s , pos_state, UserID, Data, action_space_withInx, dis_trueRcomme, pos_state_All_epoch, Pos_Neg_state_Data,
            Pos_Neg_state_Lable, All_Pos_Neg_state_Data, All_Pos_Neg_state_Lable):

  rKitems = 0
  size_a = a.size

  for i_a in range (size_a) :
    aaa = Data[int(a[i_a]), 0:num_Features + 1]

    action_space_withInx = np.delete(action_space_withInx, int(Inx_I_2[i_a]), axis=0)

    recommData = Data[int(a[i_a]) , :]
    rKitems = rKitems + recommData[-1]
    r = recommData[-1]

    if recommData[num_Features] == UserID:

      pos_state = pos_state.reshape(-1)
      pos_state = np.append(pos_state, recommData[0:num_Features], axis=0)
      pos_state = np.reshape(pos_state, (-1 , num_Features))

      pos_state_All_epoch = pos_state_All_epoch.reshape(-1)
      pos_state_All_epoch = np.append(pos_state_All_epoch, recommData[0:num_Features], axis=0)
      pos_state_All_epoch = np.reshape(pos_state_All_epoch, (-1 , num_Features))

      Pos_Neg_state_Data = Pos_Neg_state_Data.reshape(-1)
      Pos_Neg_state_Data = np.append(Pos_Neg_state_Data, recommData[0:num_Features], axis=0)
      Pos_Neg_state_Data = np.reshape(Pos_Neg_state_Data, (-1 , num_Features))

      Pos_Neg_state_Lable = np.append(Pos_Neg_state_Lable, r)

      #--------------------------------------
      All_Pos_Neg_state_Data = All_Pos_Neg_state_Data.reshape(-1)
      All_Pos_Neg_state_Data = np.append(All_Pos_Neg_state_Data, recommData[0:num_Features], axis=0)
      All_Pos_Neg_state_Data = np.reshape(All_Pos_Neg_state_Data, (-1 , num_Features))

      All_Pos_Neg_state_Lable = np.append(All_Pos_Neg_state_Lable, r)

      All_Pos_Neg_state_Data, indices = np.unique(All_Pos_Neg_state_Data, return_index=True, axis=0)
      All_Pos_Neg_state_Lable = All_Pos_Neg_state_Lable[indices]
      #--------------------------------------

      s_ = (( recommData[0:num_Features]) + selfAttention_representation( recommData[0:num_Features], pos_state))/2

    else:
      s_ = representation_state(pos_state)  # + 0.001 * s

      Pos_Neg_state_Data = Pos_Neg_state_Data.reshape(-1)
      Pos_Neg_state_Data = np.append(Pos_Neg_state_Data, recommData[0:num_Features], axis=0)
      Pos_Neg_state_Data = np.reshape(Pos_Neg_state_Data, (-1 , num_Features))

      Pos_Neg_state_Lable = np.append(Pos_Neg_state_Lable, r)

      #--------------------------------------
      All_Pos_Neg_state_Data = All_Pos_Neg_state_Data.reshape(-1)
      All_Pos_Neg_state_Data = np.append(All_Pos_Neg_state_Data, recommData[0:num_Features], axis=0)
      All_Pos_Neg_state_Data = np.reshape(All_Pos_Neg_state_Data, (-1 , num_Features))

      All_Pos_Neg_state_Lable = np.append(All_Pos_Neg_state_Lable, r)

      #print(All_Pos_Neg_state_Data.shape)

      All_Pos_Neg_state_Data, indices = np.unique(All_Pos_Neg_state_Data, return_index=True, axis=0)
      All_Pos_Neg_state_Lable = All_Pos_Neg_state_Lable[indices]
      #--------------------------------------

  return (s_ , rKitems, pos_state, action_space_withInx, dis_trueRcomme, pos_state_All_epoch, Pos_Neg_state_Data, Pos_Neg_state_Lable,
          All_Pos_Neg_state_Data, All_Pos_Neg_state_Lable)



def changeLR(epoch):
  if epoch < 50:
    return 0.01
  elif epoch< 200:
    return 0.001
  elif epoch< 400:
    return 0.001
  else:
    return 0.0001



# simple NN Generator
def ANN2(input_shape,layer_sizes, hidden_activation='relu', output_activation=None, dropout=0.2):
    inp = Input(input_shape)
    x = inp
    for layer_size in layer_sizes[:-1]:
        x = Dense(layer_size, activation='relu')(x)
        if dropout:
            x = Dropout(dropout)(x)
    #x = Dense(input_shape)(x)
    #x = Add()([x, inp])  # Skip connection
    x = Concatenate()([x, inp])  # Skip connection
    x = Activation(activation='relu')(x)
    out = Dense(layer_sizes[-1], activation=output_activation)(x)
    model = Model(inp, out)

    return model


def get_score_Lasso(weights, action_space_withInx, state_dim):
    a_space = action_space_withInx[:, :state_dim]
    scorItems = np.inner(weights, a_space)

    sortIII = np.argsort(scorItems)
    Inx_I = sortIII[scorItems.size-(850):]

    Inx_Rec_I = action_space_withInx[Inx_I, state_dim ]
    return Inx_Rec_I, Inx_I, action_space_withInx

def get_score(weights, action_space_withInx, state_dim):
    a_space = action_space_withInx[:, :state_dim]
    scorItems = np.inner(weights, a_space)  # ret = np.dot(weights, embedding.T)
    #Inx_I = np.argmax(scorItems)

    sortIII = np.argsort(scorItems)
    #print(sortIII)
    #sortVVV = np.sort(scorItems)
    Inx_I = sortIII[scorItems.size-(K_items):]
    #print("Inx_I************")
    #print(Inx_I)

    Inx_Rec_I = action_space_withInx[Inx_I, state_dim ]
    return Inx_Rec_I, Inx_I, action_space_withInx


def get_action(s, noise_scale, mu , action_max, num_actions):
    a = action_max * mu.predict(s.reshape(1,-1))[0]
    return a

def ddpg_1(
    mu_lr,
    q_lr,
    decay=0.99,
    gamma=gam,
    batch_size=bz,
    action_noise=0.0,
    num_states = num_Features,
    num_actions = num_Features):

  # Network parameters
  X_shape = (num_states)
  QA_shape = (num_states + num_actions)

  # Main network outputs
  #mu_1 = create_vit_Net(X_shape, transformer_layers, projection_dim, num_heads, transformer_units, list(hidden_sizes_1)+[num_actions], hidden_activation='relu', output_activation='tanh' ) #mu_1 = ANN2(X_shape,list(hidden_sizes_1)+[num_actions], hidden_activation='relu', output_activation='tanh')
  mu_1 = ANN2(X_shape,list(hidden_sizes_1)+[num_actions], hidden_activation='relu', output_activation='tanh')
  q_mu_1 = ANN2(QA_shape, list(hidden_sizes_2)+[1], hidden_activation='relu')

  # Target networks
  #mu_target_1 = create_vit_Net(X_shape, transformer_layers, projection_dim, num_heads, transformer_units, list(hidden_sizes_1)+[num_actions], hidden_activation='relu', output_activation='tanh' ) #mu_1 = ANN2(X_shape,list(hidden_sizes_1)+[num_actions], hidden_activation='relu', output_activation='tanh')
  mu_target_1 = ANN2(X_shape,list(hidden_sizes_1)+[num_actions], hidden_activation='relu', output_activation='tanh')
  q_mu_target_1 = ANN2(QA_shape, list(hidden_sizes_2)+[1], hidden_activation='relu')

  # Copying weights in,
  mu_target_1.set_weights(mu_1.get_weights())
  q_mu_target_1.set_weights(q_mu_1.get_weights())

  # Train each network separately
  mu_optimizer_1 =tf.keras.optimizers.Adam(learning_rate=mu_lr)
  q_optimizer_1 = tf.keras.optimizers.Adam(learning_rate=q_lr)

  return mu_1, q_mu_1, mu_target_1, q_mu_target_1, mu_optimizer_1, q_optimizer_1

def Actor_Critic_optimization(mu, q_mu, mu_target, q_mu_target, mu_optimizer, q_optimizer , X , X2, A, R, D, action_max, q_losses, mu_losses, gamma, decay):
  #Actor optimization
  with tf.GradientTape() as tape2:
    Aprime = action_max * mu(X)
    temp = tf.keras.layers.concatenate([X,Aprime],axis=1)  ## x = Concatenate()([u, m])  ## x = Dot(axes=1)([P,Q])
    #print( " temp = ")
    #print(temp)
    Q = q_mu(temp)
    mu_loss =  -tf.reduce_mean(Q)
    grads_mu = tape2.gradient(mu_loss,mu.trainable_variables)
  mu_losses.append(mu_loss)
  mu_optimizer.apply_gradients(zip(grads_mu, mu.trainable_variables))

  #Critic Optimization
  with tf.GradientTape() as tape:
    next_a = action_max * mu_target(X2)
    temp = np.concatenate((X2,next_a),axis=1)
    #print( " temp = ")
    #print(temp)
    #q_target = R + gamma * (1 - D) * q_mu_target(temp)
    q_target = R + gamma * q_mu_target(temp)
    #print(" A =")
    #print(A)
    #print(" X = ")
    #print(X)
    temp2 = np.concatenate((X,A),axis=1)
    #print( " temp2 = ")
    #print(temp2)
    qvals = q_mu(temp2)
    q_loss = tf.reduce_mean((qvals - q_target)**2)
    grads_q = tape.gradient(q_loss,q_mu.trainable_variables)
  q_optimizer.apply_gradients(zip(grads_q, q_mu.trainable_variables))
  q_losses.append(q_loss)

  ## Updating both netwokrs
  ## updating Critic network

  temp1 = np.array(q_mu_target.get_weights())
  temp2 = np.array(q_mu.get_weights())
  temp3 = decay*temp1 + (1-decay)*temp2
  q_mu_target.set_weights(temp3)

  # updating Actor network
  temp1 = np.array(mu_target.get_weights())
  temp2 = np.array(mu.get_weights())
  temp3 = decay*temp1 + (1-decay)*temp2
  mu_target.set_weights(temp3)

  return mu, q_mu, mu_target, q_mu_target, mu_optimizer, q_optimizer, q_losses, mu_losses



def switchlr(lr):
  if lr < 0.000001:
    lr = lr / 10

  return lr




import sys
sys.path.insert(0, '/content/gdrive/MyDrive/Colab Notebooks')

# Import your module or file
from buffer import *



def Main_Loop(
    num_train_episodes=n_t_e,
    gamma=gam,
    switch_rate =s_r,
    decay=0.99,
    mu_lr=lr,
    q_lr=lr,
    replay_size=replay_sizeee,
    batch_size=bz,
    start_steps=start_step,
    action_noise=0.0,
    max_episode_length=mel):

  # get size of state space and action space

  state_dim = num_Features    # state_dim = 79
  action_space = mTrain[:, :state_dim]   # action_space = mTrain[:, :state_dim]

  n_actions = len(action_space)
  observation_space = mTrain[:, :state_dim]
  n_features = num_Features  # n_features = 79

  num_states = state_dim # num_states = env.observation_space.shape[0]
  num_actions = n_features # env.action_space.shape[0]
  action_max = 1 # action_max = env.action_space.high[0] #################################################*****

  mu_2, q_mu_2, mu_target_2, q_mu_target_2, mu_optimizer_2, q_optimizer_2 = ddpg_1(gamma=gam, decay=0.99, mu_lr=lr, q_lr=lr, batch_size=bz, action_noise=0.0, num_states = num_states, num_actions = num_actions)
  mu_2_Best, q_mu_2_Best, mu_target_2_Best, q_mu_target_2_Best, mu_optimizer_2, q_optimizer_2 = ddpg_1(gamma=gam, decay=0.99, mu_lr=lr, q_lr=lr, batch_size=bz, action_noise=0.0, num_states = num_states, num_actions = num_actions)


  # Experience replay memory
  replay_buffer = BasicBuffer_b(size=replay_size,obs_dim=num_states, act_dim=num_actions)


  # Main loop: play episode and train
  returns = []
  q_losses = []
  mu_losses = []

  returns_simple = []
  q_losses_simple = []
  mu_losses_simple = []
  num_steps = 0

  Best_episode_return = -100
  Best_episode_Number = 1
  i_episode_switch = 1

  switch_flag = 0

  dis_trueRcomme = []

  s , indx = resetFun(mTrain , UserID, state_dim)  # s, episode_return, episode_length, d = resetFun(mTrain , UserID, state_dim), 0, 0, False # env.reset(), 0, 0, False
  pos_state_All_epoch = s
  count_pos_state_All_epoch = []


  max_pos_state_2 = -1
  Best_lasso_coef = np.ones(n_features)

  lasso_coef = np.ones(num_Features)*1

  # --------------------------
  s , indx = resetFun(mTrain , UserID, state_dim)
  All_Pos_Neg_state_Data = s
  All_Pos_Neg_state_Lable = mTrain[indx, num_Features]
  #---------------------------

  for i_episode in range(num_train_episodes):

    mu_lr=changeLR(i_episode)
    q_lr = changeLR(i_episode)

    # reset env
    action_space = mTrain[:, :state_dim]
    arrIndex = np.array([[i for i in range(1100)] for j in range(1)])
    action_space_Inx = np.append(action_space, arrIndex.T, axis=1)
    action_space_withInx = action_space_Inx
    action_space_withInx_1 = action_space_withInx
    action_space_withInx_2 = action_space_withInx

    episode_return_1 = 0
    episode_return_2 = 0
    episode_length = 0

    s , indx = resetFun(mTrain , UserID, state_dim)  # s, episode_return, episode_length, d = resetFun(mTrain , UserID, state_dim), 0, 0, False # env.reset(), 0, 0, False
    pos_state_2 = s
    s_2 = s

    Inx_Lasso_Recomme, Inx_I_2_2, action_space_withInx_2 = get_score_Lasso(lasso_coef, action_space_withInx_2, num_Features)
    #print("   Lasso Recomme :  ")
    #print(Inx_Lasso_Recomme)
    Inx_Lasso_Recomme_int = [0] * np.size(Inx_Lasso_Recomme)
    for u in range(np.size(Inx_Lasso_Recomme)):
      Inx_Lasso_Recomme_int[u] = int(Inx_Lasso_Recomme[u])
    selectLassoData = action_space_withInx_2[Inx_Lasso_Recomme_int , :]

    #print("Inx_Lasso_Recomme  === ")
    #print(Inx_Lasso_Recomme)

    Pos_Neg_state_Data = s
    Pos_Neg_state_Lable = mTrain[indx, num_Features]

    while not ((episode_length == max_episode_length)):  #while not (d or (episode_length == max_episode_length)):
      # For the first `start_steps` steps, use randomly sampled actions
      # in order to encourage exploration.

      if num_steps > start_steps:
        a_2 = get_action(s_2, action_noise, mu_2 , action_max , num_actions)
      else:
        a_2 = action_space[random.randrange(1,1100), :]

      # Keep track of the number of steps done
      num_steps += 1
      if num_steps == start_steps:
        print("USING AGENT ACTIONS NOW")


      # Step the env
      Inx_Rec_I_2, Inx_I_2, selectLassoData = get_score(a_2, selectLassoData, state_dim)
      # Inx_Rec_I, selectLassoData = get_score_RecommendedItems(a, selectLassoData, state_dim)

      (s2_2, r_2 , pos_state_2, selectLassoData, dis_trueRcomme, pos_state_All_epoch, Pos_Neg_state_Data,Pos_Neg_state_Lable,
       All_Pos_Neg_state_Data, All_Pos_Neg_state_Lable) = stepFun( Inx_Rec_I_2, Inx_I_2 , s_2 ,pos_state_2, UserID, Data_Reward_Train,
                                                                  selectLassoData, dis_trueRcomme, pos_state_All_epoch,Pos_Neg_state_Data,
                                                                  Pos_Neg_state_Lable, All_Pos_Neg_state_Data, All_Pos_Neg_state_Lable)


      episode_return_2 += r_2
      episode_length += 1

      d_store = 0  ## in oure problam, there isn't fainal state.

      '''# Store experience to replay buffer
      replay_buffer_simple.append([s_2, a_2, r_2, s2_2, d_store])'''

      # Store experience to replay buffer
      replay_buffer.push(s_2, a_2, r_2, s2_2, d_store)


      # Assign next state to be the current state on the next round
      s_2 = s2_2

    print("pos_state=")
    print(pos_state_2.shape)

    print("Pos_Neg_state_Data=")
    print(Pos_Neg_state_Data.shape)

    print("All_Pos_Neg_state_Data=")
    print(All_Pos_Neg_state_Data.shape)

    #--------------------------------------------------------------------
    from sklearn.linear_model import Lasso
    lasso = Lasso(alpha = 0.001)  # lasso = Lasso(alpha=0.1, normalize=True)
    lasso.fit(All_Pos_Neg_state_Data, All_Pos_Neg_state_Lable)
    lasso_coef = lasso.coef_

    pos_state_2_array = pos_state_2.reshape(-1)
    nP = pos_state_2_array.size

    if nP > max_pos_state_2:
      max_pos_state_2 = nP
      Best_lasso_coef = lasso_coef

    #--------------------------------------------------------------------
    pos_state_All_epoch = np.unique(pos_state_All_epoch, axis=0)
    #print("pos_state_All_epoch=")
    #print(pos_state_All_epoch.shape)
    num_pos , len_pos = pos_state_All_epoch.shape
    #print("num_pos  =   ", num_pos)
    count_pos_state_All_epoch.append(num_pos)

    if (episode_length % 5 == 0):
        #print("Update")
        # Perform the updates

        for _ in range(episode_length):

          X,A,R,X2,D = replay_buffer.sample(batch_size)
          X = np.asarray(X,dtype=np.float32)
          A = np.asarray(A,dtype=np.float32)
          R = np.asarray(R,dtype=np.float32)
          X2 = np.asarray(X2,dtype=np.float32)
          D = np.asarray(D,dtype=np.float32)

          '''mini_batch_simple = random.sample(replay_buffer_simple, batch_size)
          X_simple = np.array([transition[0] for transition in mini_batch_simple])
          X2_simple = np.array([transition[3] for transition in mini_batch_simple])
          A_simple = np.array([transition[1] for transition in mini_batch_simple])
          R_simple = np.array([transition[2] for transition in mini_batch_simple])
          D_simple = np.array([transition[4] for transition in mini_batch_simple])'''
          mu_2, q_mu_2, mu_target_2, q_mu_target_2, mu_optimizer_2, q_optimizer_2, q_losses_simple, mu_losses_simple = Actor_Critic_optimization(mu_2, q_mu_2, mu_target_2, q_mu_target_2 , mu_optimizer_2, q_optimizer_2, X, X2, A, R, D, action_max, q_losses_simple, mu_losses_simple, gamma,decay)
          #
    print("mu_lr = ", mu_lr)
    print("Episode:", i_episode + 1, "Return_simple  :  ", episode_return_2, 'episode_length:', episode_length)
    returns_simple.append(episode_return_2)


  return ( lasso_coef, Best_lasso_coef, max_pos_state_2, s_2, returns_simple,q_losses_simple, mu_losses_simple, mu_2, q_mu_2, mu_target_2, q_mu_target_2,episode_return_2,
          Best_episode_Number, Best_episode_return, mu_2_Best, q_mu_2_Best, mu_target_2_Best, q_mu_target_2_Best, pos_state_2,
          replay_buffer, dis_trueRcomme, count_pos_state_All_epoch, All_Pos_Neg_state_Data, All_Pos_Neg_state_Lable)




import matplotlib.pyplot as plt
def smooth(x):
      # last 100
  n = len(x)
  y = np.zeros(n)
  for i in range(n):
    start = max(0, i - 99)
    y[i] = float(x[start:(i+1)].sum()) / (i - start + 1)
  return y

(lasso_coef ,Best_lasso_coef, max_pos_state_2, s_2 , returns_simple,q_losses_simple, mu_losses_simple, mu_2, q_mu_2, mu_target_2, q_mu_target_2,episode_return_2,Best_episode_Number,Best_episode_return,
 mu_2_Best, q_mu_2_Best, mu_target_2_Best, q_mu_target_2_Best, pos_state_2, replay_buffer, dis_trueRcomme, count_pos_state_All_epoch, All_Pos_Neg_state_Data, All_Pos_Neg_state_Lable) = Main_Loop(num_train_episodes=n_t_e) # returns, q_losses,mu_losses = ddpg(lambda : gym.make('Pendulum-v0'),num_train_episodes=50)

plt.plot(returns_simple , 'g')
plt.plot(smooth(np.array(returns_simple)))
plt.title("Train returns")
plt.show()

# Plotting graph
# Episodes versus Avg. Rewards
avg_reward_list = returns_simple
plt.plot(avg_reward_list)
plt.xlabel("Episode")
plt.ylabel("Avg. Epsiodic Reward")
plt.show()

plt.plot(q_losses_simple)
plt.title('q_losses')
plt.show()

plt.plot(mu_losses_simple)
plt.title('mu_losses')
plt.show()




In [ ]:
def resetFunTest(pos_state):
  size_pos = pos_state.size
  if size_pos>num_Features :
    mean_pos = np.mean(pos_state, axis = 0)
  elif size_pos==num_Features:
    mean_pos = pos_state
  init_state = mean_pos
  return init_state


from sklearn.linear_model import Lasso
lasso = Lasso(alpha = 0.001)  # lasso = Lasso(alpha=0.1, normalize=True)
lasso.fit(All_Pos_Neg_state_Data, All_Pos_Neg_state_Lable)
lasso_coef_All = lasso.coef_



def get_score_Lasso(weights, action_space_withInx, state_dim):
    a_space = action_space_withInx[:, :state_dim]
    scorItems = np.inner(weights, a_space)

    sortIII = np.argsort(scorItems)
    #print(sortIII)
    #sortVVV = np.sort(scorItems)
    Inx_I = sortIII[scorItems.size-(100):]
    #print("Inx_I************")
    #print(Inx_I)

    Inx_Rec_I = action_space_withInx[Inx_I, state_dim ]
    # Delete row at index Inx_I
    #print("Inx_Rec_I")
    #print(Inx_Rec_I)
    #action_space_withInx = np.delete(action_space_withInx, Inx_I, axis=0)
    return Inx_Rec_I, Inx_I, action_space_withInx


def ddpgTest(
    mu = mu_2,
    q_mu = q_mu_2,
    mu_target = mu_target_2,
    q_mu_target = q_mu_target_2,
    pos_state = pos_state_2,
    replay_buffer = replay_buffer,
    num_test_episodes=1,
    gamma=gam,
    decay=0.99,
    mu_lr=lr,
    q_lr=lr,
    batch_size=bz,
    action_noise=0.0,
    max_episode_length=1):

  # get size of state space and action space

  state_dim = num_Features
  action_space = mTest[:, :state_dim]
  n_actions = len(action_space)
  observation_space = mTest[:, :state_dim]
  n_features = num_Features

  num_states = state_dim # num_states = env.observation_space.shape[0]
  num_actions = n_features # env.action_space.shape[0]
  action_max = 1 # action_max = env.action_space.high[0] #################################################*****

  # Network parameters
  X_shape = (num_states)
  QA_shape = (num_states + num_actions)

  # Main network outputs
  mu_test = ANN2(X_shape,list(hidden_sizes_1)+[num_actions], hidden_activation='relu', output_activation='tanh')
  q_mu_test = ANN2(QA_shape, list(hidden_sizes_2)+[1], hidden_activation='relu')

  # Target networks
  mu_target_test = ANN2(X_shape,list(hidden_sizes_1)+[num_actions], hidden_activation='relu', output_activation='tanh')
  q_mu_target_test = ANN2(QA_shape, list(hidden_sizes_2)+[1], hidden_activation='relu')

  # Copying weights in,
  mu_test.set_weights(mu.get_weights())
  q_mu_test.set_weights(q_mu.get_weights())
  mu_target_test.set_weights(mu_target.get_weights())
  q_mu_target_test.set_weights(q_mu_target.get_weights())

  # Train each network separately
  mu_optimizer =tf.keras.optimizers.Adam(learning_rate=mu_lr)
  q_optimizer = tf.keras.optimizers.Adam(learning_rate=q_lr)

  def get_score_RecommendedItems(weights, action_space_withInx, state_dim):
    a_space = action_space_withInx[:, :state_dim]
    scorItems = np.inner(weights, a_space)  # ret = np.dot(weights, embedding.T)
    Inx_I = np.argsort(scorItems)  # Inx_I = np.argmax(scorItems)
    Inx_Rec_I = action_space_withInx[Inx_I, state_dim ]
    # Delete row at index Inx_I
    action_space_withInx = np.delete(action_space_withInx, Inx_I, axis=0)
    return Inx_Rec_I, action_space_withInx

  def get_action(s, noise_scale):
    a = action_max * mu_test.predict(s.reshape(1,-1))[0]
    #a += noise_scale * np.random.randn(num_actions)
    #a = np.clip(a, -action_max, action_max)
    return a

  # Main loop: play episode and train
  returns_test = []
  q_losses_test = []
  mu_losses_test = []
  Matrix_Recomm_Index = []
  num_steps = 0
  for i_episode in range(num_test_episodes):

    # reset env
    action_space_B = mTest[:, :state_dim]
    arrIndex = np.array([[i for i in range(1100)] for j in range(1)])
    action_space_Inx_B = np.append(action_space_B, arrIndex.T, axis=1)
    action_space_withInx_B = action_space_Inx_B

    Inx_Lasso_Recomme, Inx_I_2, action_space_withInx_B = get_score_Lasso(lasso_coef_All, action_space_withInx_B, state_dim)
    #print("   Lasso Recomme :  ")
    #print(Inx_Lasso_Recomme)
    Inx_Lasso_Recomme_int = [0] * np.size(Inx_Lasso_Recomme)
    for u in range(np.size(Inx_Lasso_Recomme)):
      Inx_Lasso_Recomme_int[u] = int(Inx_Lasso_Recomme[u])
    selectLassoData = action_space_withInx_B[Inx_Lasso_Recomme_int , :]

    s, episode_return_test, episode_length = resetFunTest(pos_state), 0, 0  # s, episode_return, episode_length, d = resetFun(mTrain , UserID, state_dim), 0, 0, False # env.reset(), 0, 0, False

    while not ((episode_length == max_episode_length)):  #while not (d or (episode_length == max_episode_length)):

      a = get_action(s, action_noise)
      print("episode_length")
      print(episode_length)
      episode_length = episode_length + 1

      # Keep track of the number of steps done
      num_steps += 1

      # Step the env
      Inx_Rec_I, selectLassoData = get_score_RecommendedItems(a, selectLassoData, state_dim)

      print(Inx_Rec_I)

  return (Inx_Rec_I)


import matplotlib.pyplot as plt
def smooth(x):
      # last 100
  n = len(x)
  y = np.zeros(n)
  for i in range(n):
    start = max(0, i - 99)
    y[i] = float(x[start:(i+1)].sum()) / (i - start + 1)
  return y

Matrix_Recomm_Index = ddpgTest(mu_2, q_mu_2, mu_target_2, q_mu_target_2, pos_state_2, replay_buffer , num_test_episodes=1,gamma=gam,decay=0.99,mu_lr=lr,q_lr=lr,batch_size=bz,action_noise=0.0)

print("Matrix_Recomm_Index[85:100]")
print(Matrix_Recomm_Index[85:100])
print("Matrix_Recomm_Index[0:100]")
print(Matrix_Recomm_Index[0:100])




count = 0
for i in range (15):
  j = 85+i
  if (Matrix_Recomm_Index[j]<100):
    count = count +1

print("num true in 15 top_K")
print(count)

count = 0
for i in range (100):
  j = 0+i
  if (Matrix_Recomm_Index[j]<100):
    count = count +1

print("num true in 100 top_K")
print(count)



#%cd /content/gdrive/MyDrive/NWNew_RB_SPEV_DDPG_lasso_coef_intrain850_intest100
#io.savemat("NWNew_RB_SPEV_DDPG_lasso_coef_intrain850_intest100_U1.mat", {"array": Matrix_Recomm_Index})


